# IPython interface for running GPU DSM + fitting on cluster

In [8]:
from dsm_tools import ssh_init, job_start, job_done, job_transfer_clean, uuid, os, server, username, password, Calculation, CalculationStar, CalculationLinear
from fdt_fit import fdt_fit, fdt, fdtvec
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [31]:
def save(calc, filename):
    f = open(filename, 'wb')
    pickle.dump(calc, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()
def load(filename):
    f = open(filename, 'rb')
    calc = pickle.load(f)
    f.close()
    return calc

In [2]:
class IterationChain():
    def add(self, calcs):
        self.chain.append(calcs)

    def __init__(self, w):
        self.w = w
        self.chain = []

In [3]:
def mix_spectra(w, x, y):
    xWeight = w #Fraction of linear
    yWeight = 1 - w

    #Calculate p_eq of the mix
    mix_lambdaArr = np.append(x.lambdaArr, y.lambdaArr)
    mix_gArr = np.append(np.multiply(x.gArr, x.lambdaArr)/np.dot(x.gArr, x.lambdaArr) * xWeight, np.multiply(y.gArr, y.lambdaArr)/np.dot(y.gArr, y.lambdaArr) * yWeight)

    #Sort by relaxation times tau
    sort = np.argsort(mix_lambdaArr)
    mix_lambdaArr = mix_lambdaArr[sort]
    mix_gArr = mix_gArr[sort]

    return mix_lambdaArr, np.divide(mix_gArr,mix_lambdaArr)/np.sum(np.divide(mix_gArr,mix_lambdaArr))

In [4]:
nChains = 5000
CDtoggle = 0
simTime = 1000000
num_gpu = 4

#Linear chain
nk = 50

#Star chain
nArms = 3
nkArms = [6, 6, 6]

In [5]:
x, y = CalculationLinear(nk, 0, nChains, num_gpu, simTime), CalculationStar(nArms, nkArms, 0, nChains, num_gpu, simTime)

In [26]:
save(x, 'x.pkl')

In [29]:
xx = load('x.pkl')

In [0]:
weights = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0] #weight fraction of linear chains in star-linear blend
iterations = map(lambda x: IterationChain(x),weights)

#Create iteration chains for different blends
iterations[0].add((CalculationLinear(nk, 1, nChains, num_gpu, simTime)))
[i.add((CalculationLinear(nk, 1, nChains, num_gpu, simTime), CalculationStar(nArms, nkArms, 1, nChains, num_gpu, simTime))) for i in iterations[1:-1]]
iterations[-1].add((CalculationStar(nArms, nkArms, 1, nChains, num_gpu, simTime)))

To start a claculation of a **linear chain**, call `CalculationLinear(Nc, CDtoggle, Nchains, Ngpus, simTime)`

To start a claculation of a **star-branched chain**, call `CalculationStar(nArms, NcArms, CDtoggle, Nchains, Ngpus, simTime)`

### Zeroth iteration (pure linear, pure star; no constraint dynamics)

In [6]:
x.calc()
y.calc()

In [18]:
x.check_self()
y.check_self()

In [19]:
x.plot_fit_results()

In [13]:
y.plot_fit_results()

### Longest relaxation time for a pure linear and a pure star

In [20]:
#Calculate p_cr for different blends
iterations[0].chain[-1].set_pcd_cr_input(x.lambdaArr,x.gArr)

for i in iterations[1:-1]:
    a,b = mix_spectra(i.w, x, y)
    i.chain[-1][0].set_pcd_cr_input(a,b)
    i.chain[-1][1].set_pcd_cr_input(a,b)

iterations[-1].chain[-1].set_pcd_cr_input(y.lambdaArr,y.gArr)

In [21]:
#pure linear
iterations[0].chain[-1].calc()

In [77]:
iterations[0].chain[-1].check_self()

In [453]:
iterations[0].chain[-1].plot_fit_results()

In [23]:
#pure star
iterations[-1].chain[-1].calc()

In [78]:
iterations[-1].chain[-1].check_self()

In [25]:
iterations[1].chain[-1][0].calc()
iterations[1].chain[-1][1].calc()

In [99]:
iterations[1].chain[-1][0].check_self()
iterations[1].chain[-1][1].check_self()

In [458]:
iterations[1].chain[-1][0].plot_fit_results()

In [466]:
iterations[1].chain[-1][1].plot_fit_results()

In [27]:
iterations[2].chain[-1][0].calc()
iterations[2].chain[-1][1].calc()

In [100]:
iterations[2].chain[-1][0].check_self()
iterations[2].chain[-1][1].check_self()

In [29]:
iterations[3].chain[-1][0].calc()
iterations[3].chain[-1][1].calc()

In [81]:
iterations[3].chain[-1][0].check_self()
iterations[3].chain[-1][1].check_self()

In [82]:
iterations[3].chain[-1][1].plot_fit_results()

In [31]:
iterations[4].chain[-1][0].calc()
iterations[4].chain[-1][1].calc()

In [83]:
iterations[4].chain[-1][0].check_self()
iterations[4].chain[-1][1].check_self()

In [90]:
iterations[4].chain[-1][1].plot_fit_results()

In [33]:
iterations[5].chain[-1][0].calc()
iterations[5].chain[-1][1].calc()

In [95]:
iterations[5].chain[-1][0].check_self()
iterations[5].chain[-1][1].check_self()

In [96]:
iterations[5].chain[-1][1].plot_fit_results()

In [35]:
iterations[6].chain[-1][0].calc()
iterations[6].chain[-1][1].calc()

In [97]:
iterations[6].chain[-1][0].check_self()
iterations[6].chain[-1][1].check_self()

In [98]:
iterations[6].chain[-1][1].plot_fit_results()

In [45]:
iterations[7].chain[-1][0].calc()
iterations[7].chain[-1][1].calc()

In [103]:
iterations[7].chain[-1][0].check_self()
iterations[7].chain[-1][1].check_self()

In [50]:
iterations[8].chain[-1][0].calc()
iterations[8].chain[-1][1].calc()

In [109]:
iterations[8].chain[-1][0].token

In [113]:
iterations[8].chain[-1][0].check_self()
iterations[8].chain[-1][1].check_self()

In [88]:
iterations[9].chain[-1][0].calc()
iterations[9].chain[-1][1].calc()

In [116]:
iterations[9].chain[-1][0].check_self()
iterations[9].chain[-1][1].check_self()

In [122]:
iterations[9].chain[-1][0].token